In [1]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.13.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.13.0



In [2]:
!pip uninstall keras

Found existing installation: keras 2.13.1
Uninstalling keras-2.13.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/keras-2.13.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/keras/*
Proceed (Y/n)? 
  Successfully uninstalled keras-2.13.1


In [3]:
!pip install tensorflow==2.13.0

  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)


In [2]:
import scipy
import numpy
import h5py

#import tensorflow
from tensorflow import keras

#print('scipy ' + scipy.__version__)
#print('numpy ' + numpy.__version__)
#print('h5py ' + h5py.__version__)

#print('tensorflow ' + tensorflow.__version__)
#print('keras ' + keras.__version__)

import scipy.io

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Nadam
#from keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.datasets import cifar10
#error발생: from tensorflow.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical


train_x_data = scipy.io.loadmat('ml_detect_in_train.mat')
train_y_data = scipy.io.loadmat('ml_detect_out_train.mat')

train_x = train_x_data['in']
train_y = train_y_data['out']



val_x_data = scipy.io.loadmat('ml_detect_in_val.mat')
val_y_data = scipy.io.loadmat('ml_detect_out_val.mat')

val_x = val_x_data['in']
val_y = val_y_data['out']


# relu, tanh, elu, selu

model = Sequential()
model.add(Dense(units=100, input_dim=16, activation="relu", kernel_initializer="normal"))
#model.add(Dropout(0.5))
model.add(Dense(units=100, activation="relu", kernel_initializer="normal"))
#model.add(Dropout(0.5))
model.add(Dense(units=100, activation="relu", kernel_initializer="normal"))
#model.add(Dropout(0.5))
model.add(Dense(units=100, activation="relu", kernel_initializer="normal"))
#model.add(Dropout(0.5))
model.add(Dense(units=100, activation="relu", kernel_initializer="normal"))
#model.add(Dropout(0.5))
model.add(Dense(units=4, activation="linear", kernel_initializer='normal'))


#model.compile(loss='mean_squared_error', optimizer='adam')
model.compile(loss='mean_squared_error', optimizer='adamax')

#model.fit(train_x, train_y, epochs=1000, batch_size=32)

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
early_stopping = EarlyStopping(patience = 100) # 조기종료 콜백함수 정의, 100 에포크 동안은 기다림
checkpoint_callback = ModelCheckpoint('hl5_0100.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(train_x, train_y, epochs=3000, batch_size=32, validation_data=(val_x, val_y), callbacks=[early_stopping, checkpoint_callback])


from keras.models import load_model
model_cp = load_model('hl5_0100.h5')

test_x_data = scipy.io.loadmat('ml_detect_in_test.mat')
test_y_data = scipy.io.loadmat('ml_detect_out_test.mat')
test_x = test_x_data['in']
test_y = test_y_data['out']

loss_and_metrics = model_cp.evaluate(test_x, test_y, batch_size=32)

print('loss_and_metrics : ' + str(loss_and_metrics))


yhat=model_cp.predict(test_x)
scipy.io.savemat('hl5_0500_pred.mat',dict([('predict_ch', yhat) ]))


Epoch 1/3000
1/1 [==============================] - ETA: 0s - loss: 2.0909
Epoch 1: val_loss improved from inf to 2.43328, saving model to hl5_0100.h5
1/1 [==============================] - 1s 1s/step - loss: 2.0909 - val_loss: 2.4333
Epoch 2/3000
1/1 [==============================] - ETA: 0s - loss: 2.0814
Epoch 2: val_loss improved from 2.43328 to 2.42386, saving model to hl5_0100.h5
1/1 [==============================] - 0s 62ms/step - loss: 2.0814 - val_loss: 2.4239
Epoch 3/3000
1/1 [==============================] - ETA: 0s - loss: 2.0726
Epoch 3: val_loss improved from 2.42386 to 2.41367, saving model to hl5_0100.h5
1/1 [==============================] - 0s 72ms/step - loss: 2.0726 - val_loss: 2.4137
Epoch 4/3000
1/1 [==============================] - ETA: 0s - loss: 2.0630

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 4: val_loss improved from 2.41367 to 2.40235, saving model to hl5_0100.h5
1/1 [==============================] - 0s 63ms/step - loss: 2.0630 - val_loss: 2.4024
Epoch 5/3000
1/1 [==============================] - ETA: 0s - loss: 2.0523
Epoch 5: val_loss improved from 2.40235 to 2.38975, saving model to hl5_0100.h5
1/1 [==============================] - 0s 64ms/step - loss: 2.0523 - val_loss: 2.3898
Epoch 6/3000
1/1 [==============================] - ETA: 0s - loss: 2.0403
Epoch 6: val_loss improved from 2.38975 to 2.37578, saving model to hl5_0100.h5
1/1 [==============================] - 0s 70ms/step - loss: 2.0403 - val_loss: 2.3758
Epoch 7/3000
1/1 [==============================] - ETA: 0s - loss: 2.0270
Epoch 7: val_loss improved from 2.37578 to 2.36021, saving model to hl5_0100.h5
1/1 [==============================] - 0s 63ms/step - loss: 2.0270 - val_loss: 2.3602
Epoch 8/3000
1/1 [==============================] - ETA: 0s - loss: 2.0122
Epoch 8: val_loss improved from 2.3